In [ ]:
# download requirements
import os
import numpy as np
import pandas as pd

In [ ]:
# prompt for file name to convert to csv
my_file = input("File to convert to csv:")

In [ ]:
# convert file to csv
base = os.path.splitext(my_file)[0]
new_name = base + '.csv'
os.rename(my_file, new_name)
#print(base + '.txt')

In [ ]:
# read converted file into dataframe
df = pd.read_csv(new_name)
#df1 = pd.read_csv("mtsamples_original.csv")

In [ ]:
# compare two csv files
df1 = pd.read_csv("cities.csv")
df_1 = pd.DataFrame(df)
df_2 = pd.DataFrame(df1)
#df_1.eq(df_2)
#if schemas match, do nothing.  If they do not, prompt for whether or not to merge the two dataframes
if df_1.equals(df_2) == True:
    print("Columns match")
else:
    my_response = input("Schemas are not the same. Would you like to convert the file to appropriate format?:")
    if (my_response == "Yes"):
        df3 = pd.concat([df_1,df_2]).drop_duplicates(keep=False)
    #df3 = df_1.merge(df_2, how = 'inner', indicator=False)
        print(df3)
    else:
        print("No changes made")

#print(df_1.compare(df_2))
#print(df_1 == df_2)
#df_2.reset_index(drop=True, inplace=True)
#if df_1 == df_2:
 #   print("true")
#else:
 #   print("false")
#print(v)

In [ ]:
# store database file as dataframe and look at schema
import sqlite3
con = sqlite3.connect('NoteStore.sqlite')
df3 = pd.read_sql_query('select * from sqlite_master', con)
df_1 = pd.DataFrame(df3)
pd.io.json.build_table_schema(df_1)
#print(df_1.head())

In [ ]:
# compare 2 different schemas - one database and one csv
# store database file as dataframe 
import sqlite3
con = sqlite3.connect('NoteStore.sqlite')
df3 = pd.read_sql_query('select * from sqlite_master', con)
df_1 = pd.DataFrame(df3)
pd.io.json.build_table_schema(df_1)

# store csv file as dataframe
df2 = pd.read_csv("cities.csv")
df_2 = pd.DataFrame(df2)
pd.io.json.build_table_schema(df_2)

# combine the two dataframes - option if prompts will not be needed
df3 = pd.concat([df_1,df_2],axis=0, ignore_index=True)
pd.io.json.build_table_schema(df3)
#print(df_1.head())

In [ ]:
#look at 2 schemas
#df1 = pd.read_csv("cities.csv")
#df_1 = pd.DataFrame(df1)
#pd.io.json.build_table_schema(df_1)

In [ ]:
#df2 = pd.read_csv("mtsamples.csv")
#df_2 = pd.DataFrame(df2)
#pd.io.json.build_table_schema(df_2)

In [ ]:
# merge two similar dataframes - csv - into one dataframe
df1 = pd.read_csv("cities.csv")
df_1 = pd.DataFrame(df1)
pd.io.json.build_table_schema(df_1)
df2 = pd.read_csv("mtsamples.csv")
df_2 = pd.DataFrame(df2)
pd.io.json.build_table_schema(df_2)
#df3 = pd.concat([df_1,df_2]).drop_duplicates(keep=False)
df3 = pd.concat([df_1,df_2],axis=0, ignore_index=True)
pd.io.json.build_table_schema(df3)
#print(df3)

In [ ]:
# convert json file into csv
# convert json file into dataframe
pdObj = pd.read_json('movies.json', orient='index')

# convert dataframe to csv 
csvData = pdObj.to_csv('streaming.csv', index=False)

In [ ]:
# compare one dataframe to another
#df1 = pd.read_csv("cities.csv")
#df_1 = pd.DataFrame(df)
#df_2 = pd.DataFrame(df1)
#df_1.eq(df_2)
#if df_1.equals(df_2) == True:
 #   print("Columns match")
#else:
 #   my_response = input("Schemas are not the same. Would you like to convert for file to appropriate format?:")
  #  if (my_response == "Yes"):
  #      df3 = pd.concat([df_1,df_2]).drop_duplicates(keep=False)
    #df3 = df_1.merge(df_2, how = 'inner', indicator=False)
  #      print(df3)
  #  else:
  #      print("No changes made")

#print(df_1.compare(df_2))
#print(df_1 == df_2)
#df_2.reset_index(drop=True, inplace=True)
#if df_1 == df_2:
 #   print("true")
#else:
 #   print("false")
#print(v)

In [ ]:
# install data model
!pip install simpletransformers --use-feature=2020-resolver

In [ ]:
data = df[['transcription', 'medical_specialty']]
data = data.drop(data[data['transcription'].isna()].index)

In [ ]:
pip install stop_words

In [ ]:
import re
from stop_words import get_stop_words
import string

def clean_text(text): 
    text = text.translate(str.maketrans('', '', string.punctuation))
    newtext = ''.join([w for w in text if not w.isdigit()]) 
    REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
    newtext = newtext.lower()
    newtext = REPLACE_BY_SPACE_RE.sub('', newtext)
    STOPWORDS = set(get_stop_words('en'))
    words = newtext.split(" ")
    i = 0
    while i < len(words):
        if words[i] in STOPWORDS:
            words.pop(i)
        else:
            i += 1
    newtext = ' '.join(map(str, words))
    return newtext

In [ ]:
counts = data['medical_specialty'].value_counts()
others = [k for k,v in counts.items() if v<100]
for each_spec in others:
    data.loc[data['medical_specialty']==each_spec, 'medical_specialty'] = 'others' 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import RegexpTokenizer

data = data.rename(index = str, columns = {'transcription': 'text'})
data.loc[:, 'text'] = [clean_text(x) for x in data['text'].values]

le = LabelEncoder()
le.fit(data['medical_specialty'])
data.loc[:, 'class_label'] = le.transform(data['medical_specialty'])
tokenizer = RegexpTokenizer(r'\w+')
data["tokens"] = data["text"].apply(tokenizer.tokenize)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
def BOW(data):
    df_temp = data.copy(deep = True)
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(df_temp['text'])
    list_corpus = df_temp["text"].tolist()
    list_labels = df_temp["class_label"].tolist()
    X = count_vectorizer.transform(list_corpus)
    return X, list_labels

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf1(data, ngrams = 1):
    df_temp = data.copy(deep = True)
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, ngrams))
    tfidf_vectorizer.fit(df_temp['text'])
    list_corpus = df_temp["text"].tolist()
    list_labels = df_temp["class_label"].tolist()
    X = tfidf_vectorizer.transform(list_corpus)
    return X, list_labels

def tfidf2(data, ngrams = 2):
    df_temp = data.copy(deep = True)
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, ngrams))
    tfidf_vectorizer.fit(df_temp['text'])
    list_corpus = df_temp["text"].tolist()
    list_labels = df_temp["class_label"].tolist()
    X = tfidf_vectorizer.transform(list_corpus)
    return X, list_labels

In [ ]:
!python -m pip install -U gensim

In [ ]:
import gensim

word2vec = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_questions, generate_missing=False):
    embeddings = clean_questions['tokens'].apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
    return list(embeddings)

def w2v(data):
    
    df_temp = data.copy(deep = True)
    
    embeddings = get_word2vec_embeddings(word2vec, df_temp)
    list_labels = df_temp["class_label"].tolist()
    
    return embeddings, list_labels

In [ ]:
X_bow, y_bow = BOW(data)
X_tfidf1, y_tfidf1 = tfidf1(data)
X_tfidf2, y_tfidf2 = tfidf2(data)
X_w2v, y_w2v = w2v(data)

In [ ]:
from sklearn.model_selection import train_test_split
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow, y_bow, test_size=0.2, random_state=1)
X_train_tfidf1, X_test_tfidf1, y_train_tfidf1, y_test_tfidf1 = train_test_split(X_tfidf1, y_tfidf1, test_size=0.2, random_state=1)
X_train_tfidf2, X_test_tfidf2, y_train_tfidf2, y_test_tfidf2 = train_test_split(X_tfidf2, y_tfidf2, test_size=0.2, random_state=1)
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y_w2v, test_size=0.2, random_state=1)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, make_scorer
def get_metrics(y_test, y_predicted):  
    precision = precision_score(y_test, y_predicted, average='weighted')             
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

In [ ]:
from sklearn.linear_model import LogisticRegression
def training_logreg(X_train_log, X_test_log, y_train_log, y_test_log, preproc):
    
    clf = LogisticRegression(solver = 'saga', multi_class = 'multinomial', n_jobs = -1, random_state = 1)
    
    clf.fit(X_train_log, y_train_log)
    
    res = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
    
    y_pred = clf.predict(X_test_log)
    
    f1 = f1_score(y_pred, y_test_log, average = 'weighted')
    pres = precision_score(y_pred, y_test_log, average = 'weighted')
    rec = recall_score(y_pred, y_test_log, average = 'weighted')
    acc = accuracy_score(y_pred, y_test_log)
    
    res = res.append({'Preprocessing': preproc, 'Model': f'Logistic Regression', 'Precision': pres, 
                     'Recall': rec, 'F1-score': f1, 'Accuracy': acc}, ignore_index = True)

    return res

In [ ]:
full_result_logreg = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
full_result_logreg = full_result_logreg.append(training_logreg(X_train_bow, X_test_bow, y_train_bow, y_test_bow, "bow"), ignore_index = True)
full_result_logreg = full_result_logreg.append(training_logreg(X_train_tfidf1, X_test_tfidf1, y_train_tfidf1, y_test_tfidf1, "tfidf1"), ignore_index = True)
full_result_logreg = full_result_logreg.append(training_logreg(X_train_tfidf2, X_test_tfidf2, y_train_tfidf2, y_test_tfidf2, "tfidf2"), ignore_index = True)
full_result_logreg = full_result_logreg.append(training_logreg(X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v, "w2v"), ignore_index = True)
full_result_logreg

In [ ]:
from sklearn.naive_bayes import MultinomialNB
def training_naive(X_train_naive, X_test_naive, y_train_naive, y_test_naive, preproc):
    
    clf = MultinomialNB()
    clf.fit(X_train_naive, y_train_naive)

    res = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
    
    y_pred = clf.predict(X_test_naive)
    
    f1 = f1_score(y_pred, y_test_naive, average = 'weighted')
    pres = precision_score(y_pred, y_test_naive, average = 'weighted')
    rec = recall_score(y_pred, y_test_naive, average = 'weighted')
    acc = accuracy_score(y_pred, y_test_naive)
    
    res = res.append({'Preprocessing': preproc, 'Model': 'Naive Bayes', 'Precision': pres, 
                     'Recall': rec, 'F1-score': f1, 'Accuracy': acc}, ignore_index = True)

    return res

In [ ]:
!python -m pip install -U imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler

naive_under_sample = RandomUnderSampler(sampling_strategy = 'majority', random_state = 1)
X_bow_rus, y_bow_rus = naive_under_sample.fit_resample(X_bow, y_bow)
X_tfidf1_rus, y_tfidf1_rus = naive_under_sample.fit_resample(X_tfidf1, y_tfidf1)
X_w2v_rus, y_w2v_rus = naive_under_sample.fit_resample(X_w2v, y_w2v)

scaler = MinMaxScaler()
X_w2v_rus = scaler.fit_transform(X_w2v_rus)

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow_rus, y_bow_rus, test_size=0.2, random_state=1)
X_train_tfidf1, X_test_tfidf1, y_train_tfidf1, y_test_tfidf1 = train_test_split(X_tfidf1_rus, y_tfidf1_rus, test_size=0.2, random_state=1)
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v_rus, y_w2v_rus, test_size=0.2, random_state=1)

naive_result = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
naive_result = naive_result.append(training_naive(X_train_bow, X_test_bow, y_train_bow, y_test_bow, 'bow'), ignore_index = True)
naive_result = naive_result.append(training_naive(X_train_tfidf1, X_test_tfidf1, y_train_tfidf1, y_test_tfidf1, 'tfidf1'), ignore_index = True)
naive_result = naive_result.append(training_naive(X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v, 'w2v'), ignore_index = True)
naive_result

In [ ]:
!brew install lightgbm 

In [ ]:
import lightgbm as lgb
def training_lgbm(X_train_lgbm, X_test_lgbm, y_train_lgbm, y_test_lgbm, preproc,
                 n = 120, depth = 4, child = 2, bag = 0.9, feature = 0.9, l1 = 1, l = 0.01):
    
    if preproc == "w2v":
        d_train = lgb.Dataset(X_train_lgbm, label=y_train_lgbm)
    else:
        d_train = lgb.Dataset(X_train_lgbm.astype(np.float32), label=y_train_lgbm)

    early_stop = 500
    verbose_eval = False
    num_rounds = n

    params = {
        'objective': 'multiclass',
        'num_class': len(set(y_train_lgbm)),
        'boosting': 'gbdt', 
        'metric': 'multi_logloss',
        'max_depth': depth, 
        'max_bin': 22, 
        'bagging_fraction': bag, 
        'feature_fraction': feature, 
        'min_child_samples': child, 
        'min_child_weight': 1, 
        'learning_rate': l,
        'verbosity': -1, 
        'data_random_seed': 17,
        'lambda_l1': l1}

    model = lgb.train(params, train_set = d_train, num_boost_round = num_rounds)
    
    y_pred_proba = model.predict(X_test_lgbm.astype(np.float32), num_iteration=model.best_iteration)
    y_pred = [np.argmax(x) for x in y_pred_proba]
    
    res = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
        
    f1 = f1_score(y_pred, y_test_lgbm, average = 'weighted')
    pres = precision_score(y_pred, y_test_lgbm, average = 'weighted')
    rec = recall_score(y_pred, y_test_lgbm, average = 'weighted')
    acc = accuracy_score(y_pred, y_test_lgbm)
    
    res = res.append({'Preprocessing': preproc, 'Model': 'LightGBM', 'Precision': pres, 
                     'Recall': rec, 'F1-score': f1, 'Accuracy': acc}, ignore_index = True)

    return res

In [ ]:
from sklearn.svm import SVC
def training_svm(X_train_svm, X_test_svm, y_train_svm, y_test_svm, preproc):

    model = SVC()
    
    model.fit(X_train_svm, y_train_svm)
    
    y_pred = model.predict(X_test_svm)
    
    res = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
        
    f1 = f1_score(y_pred, y_test_svm, average = 'weighted')
    pres = precision_score(y_pred, y_test_svm, average = 'weighted')
    rec = recall_score(y_pred, y_test_svm, average = 'weighted')
    acc = accuracy_score(y_pred, y_test_svm)
    
    res = res.append({'Preprocessing': preproc, 'Model': 'LightGBM', 'Precision': pres, 
                     'Recall': rec, 'F1-score': f1, 'Accuracy': acc}, ignore_index = True)

    return res

In [ ]:
from imblearn.over_sampling import ADASYN

adasyn_over_sample = ADASYN(sampling_strategy='minority')
X_bow_a, y_bow_a = adasyn_over_sample.fit_resample(X_bow, y_bow)
X_tfidf1_a, y_tfidf1_a = adasyn_over_sample.fit_resample(X_tfidf1, y_tfidf1)
X_w2v_a, y_w2v_a = adasyn_over_sample.fit_resample(X_w2v, y_w2v)

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bow_a, y_bow_a, test_size=0.2, random_state=1)
X_train_tfidf1, X_test_tfidf1, y_train_tfidf1, y_test_tfidf1 = train_test_split(X_tfidf1_a, y_tfidf1_a, test_size=0.2, random_state=1)
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v_a, y_w2v_a, test_size=0.2, random_state=1)

adasyn_result = pd.DataFrame(columns = ['Preprocessing', 'Model', 'Precision', 'Recall', 'F1-score', 'Accuracy'])
adasyn_result = adasyn_result.append(training_svm(X_train_bow, X_test_bow, y_train_bow, y_test_bow, 'bow'), ignore_index = True)
adasyn_result = adasyn_result.append(training_svm(X_train_tfidf1, X_test_tfidf1, y_train_tfidf1, y_test_tfidf1, 'tfidf1'), ignore_index = True)
adasyn_result = adasyn_result.append(training_svm(X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v, 'w2v'), ignore_index = True)
adasyn_result

In [ ]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(X_w2v, y_w2v, test_size=0.2, random_state=1)
best_model = LogisticRegression(solver = 'saga', multi_class = 'multinomial', n_jobs = -1, random_state = 1)
best_model.fit(X_train_w2v, y_train_w2v)
y_test_pred = best_model.predict(X_test_w2v)
y_test = y_test_w2v

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title='Confusion matrix',
                          cmap=plt.cm.winter):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=20)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=10, rotation = 90)
    plt.yticks(tick_marks, classes, fontsize=10)
    
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", 
                 color="white" if cm[i, j] < thresh else "black", fontsize=12)
    
    plt.tight_layout()
    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)

    return plt

In [ ]:
classes = data.medical_specialty.unique()
cm = confusion_matrix(y_test, y_test_pred)
fig = plt.figure(figsize=(16, 16))
plot = plot_confusion_matrix(cm, classes=classes, normalize=False, 
                             title='Confusion matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
classes = data.medical_specialty.unique()
print(classification_report(y_test,y_test_pred,target_names=classes))